In [ ]:
%cd drive/MyDrive/AL3/AL-code

/content/drive/MyDrive/AL3/AL-code


In [ ]:
!pip install ktrain

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 25.3 MB 1.3 MB/s 
     |████████████████████████████████| 981 kB 24.6 MB/s 
     |████████████████████████████████| 263 kB 43.2 MB/s 
     |████████████████████████████████| 3.8 MB 29.9 MB/s 
     |████████████████████████████████| 1.3 MB 50.0 MB/s 
     |████████████████████████████████| 468 kB 50.2 MB/s 
     |████████████████████████████████| 880 kB 65.6 MB/s 
     |████████████████████████████████| 7.6 MB 45.4 MB/s 
     |████████████████████████████████| 163 kB 75.3 MB/s 
  Created wheel for ktrain: filename=ktrain-0.31.10-py3-none-any.whl size=25312982 sha256=beba5c4531ddc1d5ab4c14780dd03fbee4d21aa1f18a1a8f5d7c28ac6be8af25
  Stored in directory: /root/.cache/pip/wheels/9c/1c/1b/6df2db85720b8f5c6ea5e3ae37313cfc656f248abf910b7cfd
  Created wheel for keras-bert: filename=keras_bert-0.89.0-py3-none-any.whl size=33517 sha256=837ae94467c8a916b6513c4

# adaptive cluster - smallest margin

In [ ]:
from scipy.stats import entropy
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
from sklearn.metrics import roc_auc_score
from sklearn.cluster import KMeans

from collections import Counter
import random
from sklearn.model_selection import train_test_split
import ktrain
from ktrain import text
import os
import pickle
import numpy as np

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"]="0";
MODEL_NAME = "bert-base-cased"
# task = "CoLA"
# task = "OLID"
# task = "CARER"
# task = "MRDA"
# tasks = ["CS", "POLAR", "SUBJ"]
# tasks = ["CONAN"]
tasks = ["BTYPE"]
runs = ["RUN1", "RUN2", "RUN3"]
# Ns = [100,150,200,250,300]
Ns = [100,200,300]
# N = 200 # total number of labels allowed from train set
N_INIT = 50 # number of samples for bootstrapping classifier
# N_AL = N - N_INIT # number of samples to be distributed with active learning
N_ROUND = 50 # number of samples for each round of AL
ORIGINAL_N_ROUND = N_ROUND

# N_IC = 5 # number of initial clusters
# N_IC = 3 # number of initial clusters

N_ICs = [10]

############################ HELPER FUNCTIONS #############################################

def print_evaluation(gold_labels, predicted_labels):

    '''Prints accuracy, precision, recall, f1 score'''

    accuracy = accuracy_score(gold_labels, predicted_labels) * 100
    f1 = f1_score(gold_labels, predicted_labels, average = "macro") * 100
    recall = recall_score(gold_labels, predicted_labels, average = "macro") * 100
    precision = precision_score(gold_labels, predicted_labels, average = "macro") * 100


    a = [("accuracy",accuracy), ("precision",precision), ("recall", recall), ("f1-macro", f1)]
    for i in range (4):
        a[i] = (a[i][0], round(a[i][1],2))
    print (a)
    s = [round(accuracy, 2), round(precision, 2), round(recall, 2), round(f1, 2)]
    return s,a

def get_remaining(X_train,y_train, X_selected, y_selected):
    X_remaining, y_remaining = [],[]
    for i in range (len(X_train)):
        if X_train[i] not in X_selected:
            X_remaining.append(X_train[i])
            y_remaining.append(y_train[i])
    return X_remaining, y_remaining

def getKey(item):
    return item[0]

def get_cluster_metric(cluster_dict, C, model):

    cluster_texts = []
    cluster_labels = []
    try:
        for i in range (len(cluster_dict[C])):
            cluster_texts.append(cluster_dict[C][i][0])
            cluster_labels.append(cluster_dict[C][i][1])

        cluster_preds = predictor.predict(cluster_texts)
        print ("metric cluster : ", C)
        print(Counter(cluster_labels),Counter(cluster_preds))
        # cluster_scores = predictor.predict_proba(X_test)

        s,a = print_evaluation(cluster_labels, cluster_preds)
        metric = 100 - a[0][1]   #accuracy
    except:
        metric = "NAN"
    # metric = 100 - a[3][1]   #f1-score

    return metric

def update_clusters(cluster_dict, X_selected):
    new_clusters = {}
    for C in cluster_dict:
        for i in range (len(cluster_dict[C])):
            if cluster_dict[C][i][0] in X_selected:
                continue
            else:
                if C in new_clusters:
                    new_clusters[C].append((cluster_dict[C][i][0], cluster_dict[C][i][1], cluster_dict[C][i][2]))
                else:
                    new_clusters[C] = [(cluster_dict[C][i][0], cluster_dict[C][i][1], cluster_dict[C][i][2])]
    return new_clusters

def pick_samples(cluster_list, n):
    samples = random.sample(cluster_list,n)
    addXs = []
    addYs = []
    for x,y,v in samples:
        addXs.append(x)
        addYs.append(y)
    return (addXs, addYs)

def distribute_samples(X_selected, y_selected, cluster_dict, sample_cluster, N_ROUND):
    data_size = 0
    for C in cluster_dict:
        data_size += len(cluster_dict[C])

    cluster_ns = {}
    n_picked = 0

    # X = N_ROUND - N_IC

    for C in cluster_dict:
        cluster_ns [C] = int(min(round(N_ROUND*(len(cluster_dict[C])*1.0/data_size)), N_ROUND - n_picked))
        # cluster_ns [C] = cluster_ns[C] + 1

        n_picked += cluster_ns[C]

    print ("Proportional Distribution:", cluster_ns)
    leftover = N_ROUND - n_picked
    print ("Proportional leftover:", leftover)

    for C in cluster_ns:
        c_n = cluster_ns[C]
        sample_X, sample_y = pick_samples(cluster_dict[C], c_n)

        # to calculate accuracy/f1-score in each cluster
        for i in range (len(sample_X)):
            if C in sample_cluster:
                sample_cluster[C].append ((sample_X[i],sample_y[i]))
            else:
                sample_cluster[C] = [(sample_X[i], sample_y[i])]

        X_selected += sample_X
        y_selected += sample_y

    # randomly pick samples until leftover sample budget runs out
    X_remaining, y_remaining = get_remaining(X_train, y_train, X_selected, y_selected)
    while leftover > 0:
        rand_ind = random.randint(0, len(X_remaining)-1)
        if X_remaining[rand_ind] in X_selected:
            continue
        X_selected.append(X_remaining[rand_ind])
        y_selected.append(y_remaining[rand_ind])
        leftover -= 1

    return X_selected, y_selected, sample_cluster

def get_AL_samples(cluster, predictor, n):
    if n == 0:
        return [],[]
    cluster_vecs = []
    texts = []
    labels = []
    for i in range (len(cluster)):
        texts.append(cluster[i][0])
        labels.append(cluster[i][1])
        cluster_vecs.append(cluster[i][2])

    kmeans = KMeans(n_clusters=int(n), random_state=0).fit(cluster_vecs)
    cluster_labels = kmeans.labels_

    cluster_dict = {}
    for i in range (len(cluster_vecs)):
        if cluster_labels[i] in cluster_dict:
            cluster_dict[cluster_labels[i]].append((texts[i], labels[i], cluster_vecs[i]))
        else:
            cluster_dict[cluster_labels[i]] = [(texts[i], labels[i], cluster_vecs[i])]

    AL_sample_X = []
    AL_sample_Y = []
    for key in cluster_dict:
        cluster_texts = []
        cluster_labels = []
        for i in range (len(cluster_dict[key])):
            cluster_texts.append(cluster_dict[key][i][0])
            cluster_labels.append(cluster_dict[key][i][1])

        cluster_scores = predictor.predict_proba(cluster_texts)
        min_diff = 999999

        for i in range (len(cluster_dict[key])):
            probas = cluster_scores[i]
            sorted_probas = sorted(probas)
            max_proba = sorted_probas[-1]
            second_max_proba = sorted_probas[-2]
            diff = max_proba - second_max_proba
            if diff <= min_diff:
                min_diff = diff
                add_X = cluster_texts[i]
                add_y = cluster_labels[i]

        AL_sample_X.append(add_X)
        AL_sample_Y.append(add_y)

    return AL_sample_X, AL_sample_Y


def distribute_AL_samples(X_selected, y_selected, cluster_dict, val_cluster_dict, N_ROUND, predictor):
    # get ERROR (1 - accuracy/f1-score) for each cluster
    cluster_metrics = {}
    for C in cluster_dict:
        metric = get_cluster_metric(val_cluster_dict, C, predictor)
        cluster_metrics[C] = metric

    # allocate samples proportional to the ERROR
    metric_sum = sum(cluster_metrics.values())
    cluster_ns = {}

    n_picked = 0
    for C in cluster_metrics:
        if cluster_metrics[C] == 0.0:
            cluster_ns[C] = 0
            print ("something went wrong with metric in cluster:", C)

            continue

        cluster_ns [C] = min (int (min (round(N_ROUND* (cluster_metrics[C]/metric_sum)), N_ROUND - n_picked)), len(cluster_dict[C]))
        n_picked += cluster_ns[C]

    print ("AL distribution:", cluster_ns)
    leftover = N_ROUND - n_picked
    print ("AL leftover:", leftover)

    for C in cluster_dict:
        AL_sample_X, AL_sample_Y = get_AL_samples(cluster_dict[C], predictor, cluster_ns[C])
        X_selected += AL_sample_X
        y_selected += AL_sample_Y

    X_remaining, y_remaining = get_remaining(X_train, y_train, X_selected, y_selected)

    while leftover > 0:
        rand_ind = random.randint(0, len(X_remaining)-1)
        if X_remaining[rand_ind] in X_selected:
            continue
        X_selected.append(X_remaining[rand_ind])
        y_selected.append(y_remaining[rand_ind])
        leftover -= 1

    return X_selected, y_selected, val_cluster_dict
############################ LOAD AND PREP DATA #############################################
# task = "OLID"
for task in tasks:
    vector_type = "SBERT"
    data_directory = "../datasets/" + task + "/" + vector_type + "-vectors/"


    train_file = open(data_directory + "train.pkl", 'rb')
    train_data = pickle.load(train_file)
    train_file.close()

    test_file = open(data_directory + "test.pkl", 'rb')
    test_data = pickle.load(test_file)
    test_file.close()


    dev_file = open(data_directory + "dev.pkl", 'rb')
    dev_data = pickle.load(dev_file)
    dev_file.close()

    X_train, X_test, X_val, y_train, y_test, y_val = [],[],[],[],[],[]
    X_train_vecs, X_test_vecs, X_val_vecs = [],[],[]
    for key in train_data:
        X_train.append(train_data[key][0])
        y_train.append(train_data[key][1])
        X_train_vecs.append(train_data[key][2])

    for key in test_data:
        X_test.append(test_data[key][0])
        y_test.append(test_data[key][1])
        X_test_vecs.append(test_data[key][2])

    for key in dev_data:
        X_val.append(dev_data[key][0])
        y_val.append(dev_data[key][1])
        X_val_vecs.append(dev_data[key][2])


    for N_IC in N_ICs:
        for N in Ns:
            N_AL = N - N_INIT # number of samples to be distributed with active learning

            RUN_SUMMARY = [["Adaptive-cluster", "Acc", "Precision", "Recall", "F1"]]

            for run in runs:
                print ("Task:", task)
                print ("N:", N)
                print ("K:", N_IC)
                print ("----------DOING RUN:", run, "-----------------")

                print ("-----RUNNING INITIAL CLUSTERING------")
                kmeans = KMeans(n_clusters=N_IC, random_state=0).fit(X_train_vecs)
                labels = kmeans.labels_
                cluster_dict = {}
                for i in range (len(X_train_vecs)):
                    if labels[i] in cluster_dict:
                        cluster_dict[labels[i]].append((X_train[i], y_train[i], X_train_vecs[i]))
                    else:
                        cluster_dict[labels[i]] = [(X_train[i], y_train[i], X_train_vecs[i])]

                print ("CLUSTER DIST:")
                for C in cluster_dict:
                    print (C, len(cluster_dict[C]))


                val_cluster_labels = kmeans.predict(X_val_vecs)
                val_cluster_dict = {}

                for i in range (len(X_val_vecs)):
                    if val_cluster_labels[i] in val_cluster_dict:
                        val_cluster_dict[val_cluster_labels[i]].append((X_val[i], y_val[i], X_val_vecs[i]))
                    else:
                        val_cluster_dict[val_cluster_labels[i]] = [(X_val[i], y_val[i], X_val_vecs[i])]



                print ("-----INITIAL CLUSTERING DONE----")

                # pick samples according to NINIT for bootstrapping
                cats = list(set(y_train))
                checks = 0
                while (True):
                    X_selected, y_selected = [],[]
                    sample_cluster = {}
                    rand_inds = [i for i in range (len(X_train))]
                    rand_inds = random.sample(rand_inds,N_INIT)
                    X_selected = list(np.array(X_train)[rand_inds])
                    y_selected = list(np.array(y_train)[rand_inds])
                    flag = True
                    for y in cats:
                        if y not in y_selected:
                            flag = False
                    if flag:
                        break
                    checks += 1
                print ("rounds:", checks)
                # X_selected, y_selected, sample_cluster = distribute_samples(X_selected, y_selected, cluster_dict, sample_cluster, N_INIT)

                ################### bootstrap classifier #########################
                print ("current data size:", len(X_selected), len(y_selected))
                cats = list(set(y_selected))
                t = text.Transformer(MODEL_NAME, maxlen=100, class_names=cats)
                trn = t.preprocess_train(X_selected, y_selected)
                val = t.preprocess_test(X_val, y_val)

                model = t.get_classifier()
                learner = ktrain.get_learner(model, train_data=trn, val_data = val, batch_size=16)

                learner.fit_onecycle(8e-5, 5)
                print ("getting bootstrapped classifier")
                predictor = ktrain.get_predictor(learner.model, preproc=t)
                print (predictor.get_classes())
                print ("label dist after bootstrap:", Counter(y_selected))
                # test_preds = predictor.predict(X_test)
                # test_scores = predictor.predict_proba(X_test)
                # s,a = print_evaluation(y_test, test_preds)

                # print ("predicted dist of bootstrap:", Counter(test_preds))
                # print ("test results of bootstrap", a)
                print ("--------------------BOOTSTRAP DONE--------------------")

                ROUND = 0
                N_REM = N_AL
                # N_ROUND = ORIGINAL_N_ROUND/2
                while (N_REM > 0):

                    ROUND += 1
                    print (N_REM, N_ROUND)

                    print ("AL ROUND: ", ROUND, " (" + run + ") (AL STEP)")
                    cluster_dict = update_clusters(cluster_dict, X_selected)
                    X_selected, y_selected, _ = distribute_AL_samples(X_selected, y_selected, cluster_dict, val_cluster_dict, N_ROUND, predictor)
                    print ("label dist after AL step:", Counter(y_selected))

                    print ("current data size:", len(X_selected), len(y_selected))
                    cats = list(set(y_selected))
                    t = text.Transformer(MODEL_NAME, maxlen=100, class_names=cats)
                    trn = t.preprocess_train(X_selected, y_selected)
                    val = t.preprocess_test(X_val, y_val)

                    model = t.get_classifier()
                    learner = ktrain.get_learner(model, train_data=trn, val_data = val, batch_size=16)

                    learner.fit_onecycle(8e-5, 5)
                    print ("updating classifier")
                    predictor = ktrain.get_predictor(learner.model, preproc=t)
                    print (predictor.get_classes())

                    # test_preds = predictor.predict(X_test)
                    # test_scores = predictor.predict_proba(X_test)
                    # s,a = print_evaluation(y_test, test_preds)

                    # print ("predicted dist after ROUND " + str(ROUND) + ":", Counter(test_preds))
                    # print ("test results after ROUND " + str(ROUND) + ":", a)
                    print ("-----------------ROUND " + str(ROUND) + "DONE--------------")
                    N_REM = N_REM - N_ROUND

                test_preds = predictor.predict(X_test)
                test_scores = predictor.predict_proba(X_test)
                s,a = print_evaluation(y_test, test_preds)

                print ("predicted dist after ROUND " + str(ROUND) + ":", Counter(test_preds))
                print ("test results after ROUND " + str(ROUND) + ":", a)
                RUN_SUMMARY.append([run] + [str(x[1]) for x in a])
                with open ("../MC-predictions/" + task + "/individual-runs/MC-Adaptive-N" + str(N) + "-K" + str(N_IC) + "-I" + str(N_INIT) + "-R" + str(N_ROUND) + "-" + run + ".tsv", "w", encoding = "utf-8") as f:
                    label_dist = Counter(y_selected)
                    f.write("Label Distribution:\n")
                    for key in label_dist:
                        f.write(key+" : "+str(label_dist[key])+"\n")

                    # write metrics
                    f.write ("results:\n")
                    row = []
                    for (m,v) in a:
                        row.append(m + ":" + str(v))
                    line = "\t".join(row)+"\n"
                    f.write (line)

                    # write predictions
                    f.write("text\ttruth\tpred\tscore\n")
                    for i in range (len(X_test)):
                        f.write (X_test[i] + "\t" + y_test[i] + "\t" + test_preds[i] + "\t" + str(test_scores[i]) + "\n")


                print ("-------------------- DONE with " + run + "--------------------")

            with open ("../MC-predictions/" + task + "/MC-Adaptive-N" + str(N) + "-K" + str(N_IC) +  "-I" + str(N_INIT) + "-R" + str(N_ROUND) + ".tsv", "w") as f:
                for result in RUN_SUMMARY:
                    f.write("\t".join(result)+"\n")

Task: BTYPE
N: 100
K: 10
----------DOING RUN: RUN1 -----------------
-----RUNNING INITIAL CLUSTERING------
CLUSTER DIST:
0 971
5 1178
1 2634
6 1091
7 1259
4 1636
9 1363
3 1529
8 1286
2 1053
-----INITIAL CLUSTERING DONE----
rounds: 0
current data size: 50 50
preprocessing train...
language: en
train sequence lengths:
	mean : 10
	95percentile : 24
	99percentile : 28


/usr/local/lib/python3.7/dist-packages/ktrain/utils.py:738: UserWarning: class_names argument was ignored, as they were extracted from string labels in dataset
  "class_names argument was ignored, as they were extracted from string labels in dataset"


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 9
	95percentile : 19
	99percentile : 25




begin training using onecycle policy with max lr of 8e-05...
Epoch 1/5
4/4 [==============================] - 29s 5s/step - loss: 2.3574 - accuracy: 0.1000 - val_loss: 2.3968 - val_accuracy: 0.1065
Epoch 2/5
4/4 [==============================] - 11s 4s/step - loss: 2.2776 - accuracy: 0.1200 - val_loss: 2.3267 - val_accuracy: 0.1390
Epoch 3/5
4/4 [==============================] - 11s 4s/step - loss: 2.2048 - accuracy: 0.2000 - val_loss: 2.2733 - val_accuracy: 0.1540
Epoch 4/5
4/4 [==============================] - 11s 4s/step - loss: 2.0080 - accuracy: 0.3400 - val_loss: 2.2911 - val_accuracy: 0.0975
Epoch 5/5
4/4 [==============================] - 11s 4s/step - loss: 1.9495 - accuracy: 0.4200 - val_loss: 2.2660 - val_accuracy: 0.1320
getting bootstrapped classifier
['Business & Money', "Children's Books", 'Christian Books & Bibles', 'Cookbooks, Food & Wine', 'Crafts, Hobbies & Home', 'Health, Fitness & Dieting', 'Medical Books', 'Science & Math', 'Teen & Young Adult', 'Travel']
lab

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


label dist after AL step: Counter({'Crafts, Hobbies & Home': 28, 'Health, Fitness & Dieting': 24, 'Travel': 24, 'Science & Math': 23, 'Cookbooks, Food & Wine': 20, "Children's Books": 20, 'Medical Books': 17, 'Business & Money': 16, 'Christian Books & Bibles': 16, 'Teen & Young Adult': 12})
current data size: 200 200
preprocessing train...
language: en
train sequence lengths:
	mean : 10
	95percentile : 18
	99percentile : 25


/usr/local/lib/python3.7/dist-packages/ktrain/utils.py:738: UserWarning: class_names argument was ignored, as they were extracted from string labels in dataset
  "class_names argument was ignored, as they were extracted from string labels in dataset"


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 9
	95percentile : 19
	99percentile : 25




begin training using onecycle policy with max lr of 8e-05...
Epoch 1/5
13/13 [==============================] - 32s 1s/step - loss: 2.3181 - accuracy: 0.1150 - val_loss: 2.2515 - val_accuracy: 0.2055
Epoch 2/5
13/13 [==============================] - 14s 1s/step - loss: 2.1783 - accuracy: 0.1800 - val_loss: 2.1831 - val_accuracy: 0.2535
Epoch 3/5
13/13 [==============================] - ETA: 0s - loss: 2.0300 - accuracy: 0.3250